<div style="text-align:center;"><img src="http://www.mf-data-science.fr/images/projects/intro.jpg" style='width:100%; margin-left: auto; margin-right: auto; display: block;' /></div>

# <span style="color: #641E16">Contexte</span>
Nous allons ici développer un algorithme de Machine Learning destiné à assigner automatiquement plusieurs tags pertinents à une question posée sur le célébre site Stack overflow.     
Ce programme s'adresse principalement aux nouveaux utilisateurs, afin de leur suggérer quelques tags relatifs à la question qu'ils souhaitent poser.

### Les données sources
Les données ont été captées via l'outil d’export de données ***stackexchange explorer***, qui recense un grand nombre de données authentiques de la plateforme d’entraide.     
Elles portent sur la période 2009 / 2020 et **uniquement sur les posts "de qualité"** ayant au minimum 1 réponse, 5 commentaires, 20 vues et un score supérieur à 5.

### Objectif de ce Notebook
Dans ce Notebook, nous allons traiter la partie **data cleaning et exploration des données**. Un second notebook traitera ensuite les approches supervisées et non supervisées pour traiter la création de Tags à partir des données textuelles.     

Tous les Notebooks du projet seront **versionnés dans Kaggle mais également dans un repo GitHub** disponible à l'adresse https://github.com/MikaData57/Analyses-donnees-textuelles-Stackoverflow

# <span style="color:#641E16">Sommaire</span>
1. [Importation et description des données](#section_1)
2. [Data exploration](#section_2)
3. [Nettoyage des questions](#section_3)
4. [Nettoyage des titres](#section_4)
5. [Export du dataset nettoyé](#section_5)

In [ ]:
# Install package for PEP8 verification
!pip install pycodestyle
!pip install --index-url https://test.pypi.org/simple/ nbpep8

In [ ]:
# Install Beautifulsoup4
!pip install beautifulsoup4

In [ ]:
# Install langdetect
!pip install langdetect

In [ ]:
# Import Python libraries
import os
import time
from tqdm import tqdm
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup
from langdetect import detect
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import EnglishStemmer
import spacy
from spacy import displacy
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from sklearn.preprocessing import KBinsDiscretizer

# Library for PEP8 standard
from nbpep8.nbpep8 import pep8

In [ ]:
plt.style.use('seaborn-whitegrid')
sns.set_style("whitegrid")

In [ ]:
nltk.download('popular')

## <span style="color: #641E16" id="section_1">Importation et description des données</span>

In [ ]:
# Define path to data
path = '../input/stackoverflow-questions-filtered-2011-2021/'

# Concat all CSV datasets in one Pandas DataFrame
df_columns = pd.read_csv(path+'StackOverflow_questions_2009.csv').columns
data = pd.DataFrame(columns=df_columns)
for f in os.listdir(path):
    if("cleaned" not in f):
        temp = pd.read_csv(path+f)
        data = pd.concat([data, temp], 
                         axis=0,
                         ignore_index=True)
data.head(3)

In [ ]:
# Print full dataset infos
data.info()

In [ ]:
# Describe data
data.describe()

Le jeu de données ne compte pas de valeurs nulles. La variable Id ne compte que des valeurs uniques, nous pouvons donc l'utiliser en index :

In [ ]:
data.set_index('Id', inplace=True)

## <span style="color: #641E16" id="section_2">Data exploration</span>

Dans un premier temps, nous allons regarder l'**évolution du nombre de questions par année** dans notre jeu de données.

In [ ]:
# Convert CreationDate to datetime format
data['CreationDate'] = pd.to_datetime(data['CreationDate'])

# Grouper with 1 year delta
post_year = data.groupby(pd.Grouper(key='CreationDate',
                                    freq='1Y')).agg({'Title': 'count'})

# Plot evolution
fig = plt.figure(figsize=(15,6))
sns.lineplot(data=post_year, x=post_year.index, y='Title')
plt.axhline(post_year.Title.mean(), 
            color="r", linestyle='--',
            label="Mean of question per year : {:04d}"\
                   .format(int(post_year.Title.mean())))
plt.xlabel("Date of questions")
plt.ylabel("Number of questions")
plt.title("Number of questions evolution from 2009 to 2020",
          fontsize=18, color="#641E16")
plt.legend()
plt.show()

On remarque ici que sur nos critères de sélection, le nombre de questions posées a tendance à diminuer de manière constante depuis 2014.

Nous allons à présent vérifier la **longeur des différents titres** de la base :

In [ ]:
fig = plt.figure(figsize=(20, 12))
ax = sns.countplot(x=data.Title.str.len())
start, end = ax.get_xlim()
ax.xaxis.set_ticks(np.arange(0, end, 5))
plt.axvline(data.Title.str.len().median() - data.Title.str.len().min(),
            color="r", linestyle='--',
            label="Title Lenght median : "+str(data.Title.str.len().median()))
ax.set_xlabel("Lenght of title")
plt.title("Title lenght of Stackoverflow questions",
          fontsize=18, color="#641E16")
plt.legend()
plt.show()

Nous allons également ploter la répartition des **longueurs de la variable** `Body` *(les corps de texte des questions)*. L'étendue étant très importante, nous allons dans un premier temps **discrétiser ces longueur** pour ne pas surcharger les temps de calculs de projection graphique :

In [ ]:
# Discretizer for Body characters lenght
X = pd.DataFrame(data.Body.str.len())

# Sklearn discretizer with 200 bins
discretizer = KBinsDiscretizer(n_bins=200,
                               encode='ordinal',
                               strategy='uniform')
body_lenght = discretizer.fit_transform(X)
body_lenght = discretizer.inverse_transform(body_lenght)
body_lenght = pd.Series(body_lenght.reshape(-1))

In [ ]:
fig = plt.figure(figsize=(20, 12))
ax = sns.countplot(x=body_lenght)
start, end = ax.get_xlim()
ax.xaxis.set_ticks(np.arange(0, end, 25))
ax.set_xlabel("Lenght of Body (after discretization)")
plt.title("Body lenght of Stackoverflow questions",
          fontsize=18, color="#641E16")
plt.legend()
plt.show()

On remarque que la majeur partie des questions compte moins de 4000 caractères *(balises HTML compris)* mais certains posts dépassent les 31 000 caractères. Nous allons **filtrer notre jeu de données pour conserver uniquement les questions de moins de 4 000 caractères** afin de ne pas compliquer le NLP plus que nécessaire.

In [ ]:
# Filter data on body lenght
data = data[data.Body.str.len() < 4000]
data.shape

### Analyse des tags
Nous allons faire une rapide analyse exploratoire sur les tags du jeu de données.

In [ ]:
data['Tags'].head(3)

Nous allons modifier les séparateurs de Tags pour favoriser les extractions :

In [ ]:
# Replace open and close balise between tags
data['Tags'] = data['Tags'].str.translate(str.maketrans({'<': '', '>': ','}))

# Delete last "," for each row
data['Tags'] = data['Tags'].str[:-1]
data['Tags'].head(3)

Les tags contenus dans la variable `Tags` sont ensuite splités et ajoutés dans une liste pour ensuite les classer :

In [ ]:
def count_split_tags(df, column, separator):
    """This function allows you to split the different words contained
    in a Pandas Series cell and to inject them separately into a list.
    This makes it possible, for example, to count the occurrences of words.

    Parameters
    ----------------------------------------
    df : Pandas Dataframe
        Dataframe to use.
    column : string
        Column of the dataframe to use
    separator : string
        Separator character for str.split.
    ----------------------------------------
    """
    list_words = []
    for word in df[column].str.split(separator):
        list_words.extend(word)
    df_list_words = pd.DataFrame(list_words, columns=["Tag"])
    df_list_words = df_list_words.groupby("Tag")\
        .agg(tag_count=pd.NamedAgg(column="Tag", aggfunc="count"))
    df_list_words.sort_values("tag_count", ascending=False, inplace=True)
    return df_list_words

In [ ]:
tags_list = count_split_tags(df=data, column='Tags', separator=',')
print("Le jeu de données compte {} tags.".format(tags_list.shape[0]))

In [ ]:
# Plot the results of splits
fig = plt.figure(figsize=(15, 8))
sns.barplot(data=tags_list.iloc[0:40, :],
            x=tags_list.iloc[0:40, :].index,
            y="tag_count", color="#f48023")
plt.xticks(rotation=90)
plt.title("40 most popular tags in Stackoverflow (2009 - 2020)",
          fontsize=18, color="#641E16")
plt.show()

Dans les 40 tags les plus populaires sur StackOverflow, les tags **C++**, **C#** et **java** sont sans surprise dans le top 3. Le dataset compte **plus de 16 800 tags** différents pour la période 2009 - 2020. 

Nous pouvons également **visualiser les 500 premières catégories dans un nuage de mots** :

In [ ]:
# Plot word cloud with tags_list (frequencies)
fig = plt.figure(1, figsize=(17, 12))
ax = fig.add_subplot(1, 1, 1)
wordcloud = WordCloud(width=900, height=500,
                      background_color="black",
                      max_words=500, relative_scaling=1,
                      normalize_plurals=False)\
    .generate_from_frequencies(tags_list.to_dict()['tag_count'])

ax.imshow(wordcloud, interpolation='bilinear')
ax.axis("off")
plt.title("Word Cloud of 500 best Tags on StackOverflow (2009 - 2020)\n",
          fontsize=18, color="#641E16")
plt.show()

Il peut être intéressant de regarder si ces tags populaires ont évolués au fil du temps. Prenons par exemple les années 2009, 2012, 2016 et 2020 pour vérifier.

In [ ]:
# Subplots parameters
years = {0: 2009, 1: 2012, 2: 2016, 3: 2020}
colors = {0: "#f48023", 1: "#d16e1e",
          2: "#b25d19", 3: "#904b14"}
subplots = 4
cols = 2
rows = subplots // cols
rows += subplots % cols
position = range(1, subplots + 1)

# Plot popular tags for each year
fig = plt.figure(1, figsize=(20, 16))
for k in range(subplots):
    subset = data[data["CreationDate"].dt.year == years[k]]
    temp_list = count_split_tags(df=subset, column='Tags', separator=',')
    ax = fig.add_subplot(rows, cols, position[k])
    sns.barplot(data=temp_list.iloc[0:20, :],
            x=temp_list.iloc[0:20, :].index,
            y="tag_count", color=colors[k])
    plt.xticks(rotation=90)
    ax.set_title("20 most popular tags for {}".format(years[k]),
                 fontsize=18, color="#641E16")

fig.tight_layout()
plt.show()

On remarque en effet que les centres d'intérêt évoluent en fonction des années. Cependant, on retrouve les principaux languages et framework informatiques dans les premières places.     

Nous allons a présent regarder le **nombre de Tags par question** :

In [ ]:
# Create a list of Tags and count the number
data['Tags_list'] = data['Tags'].str.split(',')
data['Tags_count'] = data['Tags_list'].apply(lambda x: len(x))

# Plot the result
fig = plt.figure(figsize=(12, 8))
ax = sns.countplot(x=data.Tags_count, color="#f48023")
ax.set_xlabel("Tags")
plt.title("Number of tags used per question",
          fontsize=18, color="#641E16")
plt.show()

Pour la majorité des questions StackOverflow analysées, 3 tags sont utilisés. Cela nous donne déjà une indication sur le type de modélisation à mettre en oeuvre.

### Filtrage du jeu de données avec les meilleurs Tags : 
Les process de NLP sont des algorithmes assez lents compte tenu de la quantité de données à traiter. Pour filtrer notre jeu de données, nous allons **sélectionner toutes les questions qui comportent au moins 1 des 50 meilleurs tags** :

In [ ]:
top_tags = list(tags_list.iloc[0:50].index)
data['Top_tag'] = data['Tags_list'].apply(lambda x : any(item in x for item in top_tags))
data = data[data.Top_tag == True]
print("New size of dataset : {} questions.".format(data.shape[0]))

## <span style="color: #641E16" id="section_3">Nettoyage des questions</span>

Afin de traiter au mieux les données textuelles du `Body`, il est nécessaire de réaliser plusieurs tâches de data cleaning. Par exemple, le texte stocké dans cette variable est au format HTML. Ces balises vont polluer notre analyse. Nous allons donc **supprimer toutes les balises HTML** avec la librairie `BeautifulSoup` pour ne conserver que le texte brut.

Mais avant cette opération,nous allons supprimer tout le contenu placé entre 2 balises html `<code></code>`, cela nous permettra de supprimer tout le code brut souvent copié dans les questions Stackoverflow et qui pourrait avoir un fort impact pour la suite.

In [ ]:
def remove_code(x):
    """Function based on the Beautifulsoup library intended to replace 
    the content of all the <code> </code> tags of a text specified as a parameter.

    Parameters
    ----------------------------------------
    x : string
        Sequence of characters to modify.
    ----------------------------------------
    """
    soup = BeautifulSoup(x,"lxml")
    code_to_remove = soup.findAll("code")
    for code in code_to_remove:
        code.replace_with(" ")
    return str(soup)

In [ ]:
start_time = time.time()
# Delete <code> in Body text
data['Body'] = data['Body'].apply(remove_code)
# Delete all html tags
data['Body'] = [BeautifulSoup(text,"lxml").get_text() for text in data['Body']]
exec_time = time.time() - start_time
print('-' * 50)
print("Execution time : {:.2f}s".format(exec_time))
print('-' * 50)
print(data['Body'].head(3))

A présent, nous devons **vérifier si les textes des questions sont rédigés en diverses langues**. Cela nous permettra de définir la liste des stop words à éliminer :

In [ ]:
# Create feature "lang" with langdetect library
def detect_lang(x):
    try:
        return detect(x)
    except:
        pass

start_time = time.time()
data['short_body'] = data['Body'].apply(lambda x: x[0:100])
data['lang'] = data.short_body.apply(detect_lang)
exec_time = time.time() - start_time
print('-' * 50)
print("Execution time : {:.2f}s".format(exec_time))
print('-' * 50)

In [ ]:
# Count titles for each language
pd.DataFrame(data.lang.value_counts())

La langue Anglaise est très majoritairement représentée dans notre dataset. Nous allons donc **supprimer de notre jeu de données tous les post dans une autre langue que l'anglais**.

In [ ]:
# Deletion of data that is not in the English language
data = data[data['lang']=='en']

Maintenant que nous avons un texte brut débarassé de ses balises HTML et du code, nous allons utiliser `nltk.pos_tag` pour **identifier la nature de chaque mot du corpus afin de pouvoir ensuite conserver uniquement les noms**. Nous allons ici créer une function qui sera appliquée ensuite dans un cleaner plus complet *(tout aurait pu être réalisé avec SpaCy, mais pour l'exercice et la méthode, les autres étapes sont détaillées)*.

In [ ]:
def remove_pos(nlp, x, pos_list):
    doc = nlp(x)
    list_text_row = []
    for token in doc:
        if(token.pos_ in pos_list):
            list_text_row.append(token.text)
    join_text_row = " ".join(list_text_row)
    join_text_row = join_text_row.lower().replace("c #", "c#")
    return join_text_row

Nous allons à présent réaliser plusieurs opérations de Text cleaning pour que nos données soient exploitables par les algorithmes de NLP :
- Suppression de tous les mots autres que les noms
- Mettre tout le texte en **minuscules**
- Supprimer les **caractères Unicode** (comme les Emojis par exemple)
- Suppression des **espaces supplémentaires**
- Suppression de la **ponctuation**
- Suppression des **liens**
- Supprimer les **nombres**

In [ ]:
def text_cleaner(x, nlp, pos_list):
    """Function allowing to carry out the preprossessing on the textual data. 
        It allows you to remove extra spaces, unicode characters, 
        English contractions, links, punctuation and numbers.
        
        The re library for using regular expressions must be loaded beforehand.

    Parameters
    ----------------------------------------
    x : string
        Sequence of characters to modify.
    ----------------------------------------
    """
    # Remove POS not in "NOUN", "PROPN"
    x = remove_pos(nlp, x, pos_list)
    # Case normalization
    x = x.lower()
    # Remove unicode characters
    x = x.encode("ascii", "ignore").decode()
    # Remove English contractions
    x = re.sub("\'\w+", '', x)
    # Remove ponctuation but not # (for C# for example)
    x = re.sub('[^\\w\\s#]', '', x)
    # Remove links
    x = re.sub(r'http*\S+', '', x)
    # Remove numbers
    x = re.sub(r'\w*\d+\w*', '', x)
    # Remove extra spaces
    x = re.sub('\s+', ' ', x)
    
    # Return cleaned text
    return x

In [ ]:
# Apply cleaner on Body
# Spacy features
nlp = spacy.load('en', exclude=['tok2vec', 'ner', 'parser', 
                                'attribute_ruler', 'lemmatizer'])
pos_list = ["NOUN","PROPN"]

start_time = time.time()
print('-' * 50)
print("Start Body cleaning ...")
print('-' * 50)

tqdm.pandas()
data['Body_cleaned'] = data.Body.progress_apply(lambda x : text_cleaner(x, nlp, pos_list))

exec_time = time.time() - start_time
print("Execution time : {:.2f}s".format(exec_time))
print('-' * 50)
print(data['Body_cleaned'].head(3))

Nous pouvons à présent **supprimer tous les stop words en langue Anglaise** grâce à la librairie `NLTK`. Avant cette étape, nous allons réaliser une **tockenisation** c'est à dire découper les phrase en mots et création d'une liste *(chaque phrase est une liste de mots)*

In [ ]:
2+2

In [ ]:
start_time = time.time()
# Tockenization
data['Body_cleaned'] = data.Body_cleaned.apply(nltk.tokenize.word_tokenize)

# List of stop words in "EN" from NLTK
stop_words = stopwords.words("english")

# Remove stop words
data['Body_cleaned'] = data.Body_cleaned\
    .apply(lambda x : [word for word in x
                       if word not in stop_words
                       and len(word)>2])
exec_time = time.time() - start_time
print('-' * 50)
print("Execution time : {:.2f}s".format(exec_time))
print('-' * 50)
print(data['Body_cleaned'].head(3))

A présent, nous avons des listes de mots débarrassées des mots courants (stop words), de la ponctuation, des liens et des nombres. Une dernière étape que nous pouvons effectuer est la **Lemmatisation**. Ce procédé consiste à prend le mot à sa forme racine appelée Lemme. Cela nous permet d'amener les mots à leur forme "dictionnaire". Nous allons pour cela utiliser à nouveau la librairie `NLTK`.

In [ ]:
# Apply lemmatizer on Body
start_time = time.time()
wn = WordNetLemmatizer()
data['Body_cleaned'] = data.Body_cleaned\
    .apply(lambda x : [wn.lemmatize(word) for word in x])
exec_time = time.time() - start_time
print('-' * 50)
print("Execution time : {:.2f}s".format(exec_time))
print('-' * 50)
print(data['Body_cleaned'].head(3))

Le body étant à présant cleané, nous allons regarder la répartition de la **taille des corpus** dans le jeu de donné nettoyé :

In [ ]:
# Calculate lenght of each list in Body
data['body_tokens_count'] = [len(_) for _ in data.Body_cleaned]

# Countplot of body lenght
fig = plt.figure(figsize=(20, 12))
ax = sns.countplot(x=data.body_tokens_count)
start, end = ax.get_xlim()
ax.xaxis.set_ticks(np.arange(0, end, 25))
plot_median = data.body_tokens_count.median()
plt.axvline(plot_median - data.body_tokens_count.min(),
            color="r", linestyle='--',
            label="Body tokens Lenght median : "+str(plot_median))
ax.set_xlabel("Lenght of body tokens")
plt.title("Body tokens lenght of Stackoverflow questions after cleaning",
          fontsize=18, color="#641E16")
plt.legend()
plt.show()

Nous allons également regarder les **fréquences de chaque mots de la variable Body** pour visualiser les plus représentés :

In [ ]:
# Create a list of all tokens for Body
full_corpus = []
for i in data['Body_cleaned']:
    full_corpus.extend(i)

In [ ]:
# Calculate distribition of words in Body token list
body_dist = nltk.FreqDist(full_corpus)
body_dist = pd.DataFrame(body_dist.most_common(2000),
                         columns=['Word', 'Frequency'])
body_dist.describe()

On remarque que 75% des 2000 mots les plus communs apparaissent 667 fois *(3ème quartile)*. Or, le maximum se situe à plus de 45 000.

In [ ]:
# Plot word cloud with tags_list (frequencies)
fig = plt.figure(1, figsize=(17, 12))
ax = fig.add_subplot(1, 1, 1)
wordcloud = WordCloud(width=900, height=500,
                      background_color="black",
                      max_words=500, relative_scaling=1,
                      normalize_plurals=False)\
    .generate_from_frequencies(body_dist.set_index('Word').to_dict()['Frequency'])

ax.imshow(wordcloud, interpolation='bilinear')
ax.axis("off")
plt.title("Word Cloud of 500 most popular words on Body feature\n",
          fontsize=18, color="#641E16")
plt.show()

## <span style="color: #641E16" id="section_4">Nettoyage des titres</span>
Nous avons préalablement défini une fonction pour notre cleaning des Body. Nous allons la modifier pour y intégrer la tokenisation, les stop words et la lemmanisation afin d'obtenir un processus complet à appliquer aux titres des posts.

In [ ]:
def text_cleaner(x, nlp, pos_list, lang="english"):
    """Function allowing to carry out the preprossessing on the textual data. 
        It allows you to remove extra spaces, unicode characters, 
        English contractions, links, punctuation and numbers.
        
        The re library for using regular expressions must be loaded beforehand.
        The SpaCy and NLTK librairies must be loaded too. 

    Parameters
    ----------------------------------------
    x : string
        Sequence of characters to modify.
    ----------------------------------------
    """
    # Remove POS not in "NOUN", "PROPN"
    x = remove_pos(nlp, x, pos_list)
    # Case normalization
    x = x.lower()
    # Remove unicode characters
    x = x.encode("ascii", "ignore").decode()
    # Remove English contractions
    x = re.sub("\'\w+", '', x)
    # Remove ponctuation but not # (for C# for example)
    x = re.sub('[^\\w\\s#]', '', x)
    # Remove links
    x = re.sub(r'http*\S+', '', x)
    # Remove numbers
    x = re.sub(r'\w*\d+\w*', '', x)
    # Remove extra spaces
    x = re.sub('\s+', ' ', x)
        
    # Tokenization
    x = nltk.tokenize.word_tokenize(x)
    # List of stop words in select language from NLTK
    stop_words = stopwords.words(lang)
    # Remove stop words
    x = [word for word in x if word not in stop_words 
         and len(word)>2]
    # Lemmatizer
    wn = nltk.WordNetLemmatizer()
    x = [wn.lemmatize(word) for word in x]
    
    # Return cleaned text
    return x

In [ ]:
# Spacy features
nlp = spacy.load('en', exclude=['tok2vec', 'ner', 'parser', 
                                'attribute_ruler', 'lemmatizer'])
pos_list = ["NOUN","PROPN"]
# Apply full cleaner on Title
print('-' * 50)
print("Start Title cleaning ...")
print('-' * 50)
start_time = time.time()
data['Title_cleaned'] = data.Title\
                            .progress_apply(lambda x: 
                                            text_cleaner(x,
                                                         nlp,
                                                         pos_list,
                                                         "english"))
exec_time = time.time() - start_time
print("Execution time : {:.2f}s".format(exec_time))
print('-' * 50)
print(data['Title_cleaned'].head(3))

Nous pouvons à présent projeter la distribution de la taille des tokens Title et le nuage de mots correspondant aux 500 meilleurs apparitions : 

In [ ]:
# Calculate lenght of each list in Body
data['Title_tokens_count'] = [len(_) for _ in data.Title_cleaned]

# Countplot of body lenght
fig = plt.figure(figsize=(20, 12))
ax = sns.countplot(x=data.Title_tokens_count)
median_plot = data.Title_tokens_count.median()
plt.axvline(median_plot - data.Title_tokens_count.min(),
            color="r", linestyle='--',
            label="Title tokens Lenght median : "+str(median_plot))
ax.set_xlabel("Lenght of body tokens")
plt.title("Title tokens lenght of Stackoverflow questions after cleaning",
          fontsize=18, color="#641E16")
plt.legend()
plt.show()

In [ ]:
# Create a list of all tokens for Title
full_corpus_t = []
for i in data['Title_cleaned']:
    full_corpus_t.extend(i)

# Calculate distribition of words in Title token list
title_dist = nltk.FreqDist(full_corpus_t)
title_dist = pd.DataFrame(title_dist.most_common(500),
                          columns=['Word', 'Frequency'])

# Plot word cloud with tags_list (frequencies)
fig = plt.figure(1, figsize=(17, 12))
ax = fig.add_subplot(1, 1, 1)
wordcloud = WordCloud(width=900, height=500,
                      background_color="black",
                      max_words=500, relative_scaling=1,
                      normalize_plurals=False)\
    .generate_from_frequencies(title_dist.set_index('Word').to_dict()['Frequency'])

ax.imshow(wordcloud, interpolation='bilinear')
ax.axis("off")
plt.title("Word Cloud of 500 most popular words on Title feature\n",
          fontsize=18, color="#641E16")
plt.show()

## <span style="color: #641E16" id="section_5">Export du dataset nettoyé</span>
Nous pouvons maintenant supprimer les variables créées pour l'analyse exploratoire, qui ne nous seront plus utiles, et exporter le dataset pour nos modélisations supervisées et non supervisées disponible dans le Notebook [Stackoverflow questions - tag generator](https://www.kaggle.com/michaelfumery/stackoverflow-questions-tag-generator)

Enfin, afin d'avoir suffisement de "matière" pour alimenter nos algorithmes de prédiction de Tags, nous allons **conserver uniquement les questions qui comptent au minimum 5 tokens dans la variable Body**.

In [ ]:
# Delete items with number of Body tokens < 5
data = data[(data.body_tokens_count >= 5) & (data.Title_tokens_count > 0)]
# Remove calculated features
data = data[['Title_cleaned',
             'Body_cleaned',
             'Score',
             'Tags_list']]
# Rename columns
data = data.rename(columns={'Title_cleaned': 'Title',
                            'Body_cleaned': 'Body',
                            'Tags_list': 'Tags'})
data.head(3)

In [ ]:
# Export to CSV
data.to_csv("StackOverflow_questions_2009_2020_cleaned.csv", sep=";")